In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

# Generate dataset

names = ["condensed_2010.json","condensed_2011.json","condensed_2012.json","condensed_2013.json",
         "condensed_2014.json", "condensed_2015.json","condensed_2016.json","condensed_2017.json","condensed_2018.json",
         "trump2019_26_03.json"]

data = pd.read_json("condensed_2009.json")
for element in names:
    data1 = pd.read_json(element)
    print(len(data1),element)
    data = data.append(data1,sort=True)
data = data.reset_index(drop=True)


#pd.to_datetime(stamps, format="%Y-%m-%d %H:%M:%S").sort_values()

data['created_at'] = pd.to_datetime(data.created_at, format="%Y-%m-%d %H:%M:%S")
data.sort_values(by=['created_at'], inplace=True, ascending=True)
data.reset_index(inplace=True,drop=True)
data.to_json("orderedTwitterArchive.json")
data.to_csv("orderedTwitterArchive.csv")
data

# Exploration

In [ ]:
data = pd.read_json("orderedTwitterArchive.json")
data['created_at'] = pd.to_datetime(data.created_at, format="%Y-%m-%d %H:%M:%S")
data.sort_values(by=['created_at'], inplace=True, ascending=True)
data.reset_index(inplace=True,drop=True)
data

In [ ]:
fig, ax = plt.subplots(figsize = (25, 10))
ax = sns.lineplot(x=data.index, y="favorite_count", data=data) 

In [ ]:
fig, ax = plt.subplots(figsize = (25, 10))
ax = sns.lineplot(x=data.index, y="retweet_count", data=data) 

In [ ]:
data["text"][2005]

# idee

NLP:
    - pos tagging per contare numero di aggettivi 
    - qualcosa per vedere se aggettivi sono positivi o negativi (direttamente sentiment analysis)?
    - rimuovere gli @ vs tenere solo i più frequenti 
    - rimuovere i link
    - pulire gli hashtag? 
    - word2vec distance per avere simili tweet assieme nello spazio vettoriale?
    - ridurre parole presenti? 
    - tfidf?
    
Processo:
    - clustering interno all'anno
    - clustering generale 
    - clustering con/senza sentiment meaning